In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

# Define the state for user data
class UserState(TypedDict):
    is_premium: bool
    message: str

# Define nodes
def greet_user(state: UserState):
    state['message'] = "welcome to our service!"
    return state

def premium_greeting(state: UserState):
    state['message'] = "thank you for being a premium member!"
    return state

def regular_greeting(state: UserState):
    state['message'] = "consider upgrading to premium for more benefits."
    return state

# Define the decision node to choose the greeting based on user status
def check_subscription(state: UserState):
    if state['is_premium']:
        return 'premium_greeting'
    else:
        return 'regular_greeting'
    

# Build the state graph
graph_builder = StateGraph(UserState)
graph_builder.add_node("greet_user", greet_user)
graph_builder.add_node("premium_greeting", premium_greeting)
graph_builder.add_node("regular_greeting", regular_greeting)
graph_builder.add_node("check_subscription", check_subscription)


# Add edges to control flow
graph_builder.add_edge(START, "greet_user")
graph_builder.add_conditional_edges("greet_user", check_subscription)
graph_builder.add_edge( "premium_greeting", END)
graph_builder.add_edge( "regular_greeting", END)



/opt/anaconda3/envs/langchain_ai/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# Compile and run for premium user
graph = graph_builder.compile()
result = graph.invoke({'is_premium': True, 'message': ''})
print(result)


{'is_premium': True, 'message': 'thank you for being a premium member!'}


In [3]:
# Compile and run for regular user
graph = graph_builder.compile()
result = graph.invoke({'is_premium': False, 'message': ''})
print(result)

{'is_premium': False, 'message': 'consider upgrading to premium for more benefits.'}
